# Correlation analysis
- what is correlation analysis
- how does correlation analysis help with data cleaning
- coding examples

## Correlation analysis 
It is a technique used to examine the strength and direction of the relationship between two aor more variables
Use `Correlation Coefficients`.
${Y, X}$

### Popular Correlation coefficients

#### Pearson
- USe for continuous data
- Measures the strength of the `linear relationship` between the variable
- sensitive to outliers

#### Spearman
- Use for ordinal or ranked data
- Measures the strength of the `monotonic relationship` between the variables, which can be linear or non-linear
- more robust to outliers 

Correlation analysis can identified variables that are highly correlated to each other.

The analysis may indicate if one variable is `redundant` and can be eliminated.

### Dealing with categorical data
- Data has many `non-numerical` features. You CANNOT feed them into the model. THey need to be converted.
- Use the `pd.dtypes()` to see the data types

### Two main methods
- Label encoding
- One-hot encoding

### Label encoding
Each `unique` category in the categorical variable is assigned a numerical label. Typically starting at 0, 1, etc

### One-hot encoding
a new binary feature is created `for each` category, and the value of that feature is a set to 1 if the observation belong to it.

## Dealing with categorical data

In [1]:
import pandas as pd
import numpy as np

In [2]:
# 
data_df = pd.read_csv('../data/hotel_bookings.csv')
data_df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [3]:
# Create a new df from only columns with object data types
data_df.dtypes

hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                             

In [4]:
# Create list of columns with dtype  == 'object'
# use a list comprehension with predicate
cat_cols = [col for col in data_df.columns if data_df[col].dtype == 'object']
cat_cols

['hotel',
 'arrival_date_month',
 'meal',
 'country',
 'market_segment',
 'distribution_channel',
 'reserved_room_type',
 'assigned_room_type',
 'deposit_type',
 'customer_type',
 'reservation_status',
 'reservation_status_date']

In [5]:
# create a dataFrame with object columns only
cat_df = data_df[cat_cols]
cat_df.head()

,hotel,arrival_date_month,meal,country,market_segment,distribution_channel,reserved_room_type,assigned_room_type,deposit_type,customer_type,reservation_status,reservation_status_date
0,Resort Hotel,July,BB,PRT,Direct,Direct,C,C,No Deposit,Transient,Check-Out,2015-07-01
1,Resort Hotel,July,BB,PRT,Direct,Direct,C,C,No Deposit,Transient,Check-Out,2015-07-01
2,Resort Hotel,July,BB,GBR,Direct,Direct,A,C,No Deposit,Transient,Check-Out,2015-07-02
3,Resort Hotel,July,BB,GBR,Corporate,Corporate,A,A,No Deposit,Transient,Check-Out,2015-07-02
4,Resort Hotel,July,BB,GBR,Online TA,TA/TO,A,A,No Deposit,Transient,Check-Out,2015-07-03


In [6]:
# print unique values of each feature in cat_df
for col in cat_df.columns:
    print(f'{col}: \n {cat_df[col].unique()}')

hotel: 
 ['Resort Hotel' 'City Hotel']
arrival_date_month: 
 ['July' 'August' 'September' 'October' 'November' 'December' 'January'
 'February' 'March' 'April' 'May' 'June']
meal: 
 ['BB' 'FB' 'HB' 'SC' 'Undefined']
country: 
 ['PRT' 'GBR' 'USA' 'ESP' 'IRL' 'FRA' nan 'ROU' 'NOR' 'OMN' 'ARG' 'POL'
 'DEU' 'BEL' 'CHE' 'CN' 'GRC' 'ITA' 'NLD' 'DNK' 'RUS' 'SWE' 'AUS' 'EST'
 'CZE' 'BRA' 'FIN' 'MOZ' 'BWA' 'LUX' 'SVN' 'ALB' 'IND' 'CHN' 'MEX' 'MAR'
 'UKR' 'SMR' 'LVA' 'PRI' 'SRB' 'CHL' 'AUT' 'BLR' 'LTU' 'TUR' 'ZAF' 'AGO'
 'ISR' 'CYM' 'ZMB' 'CPV' 'ZWE' 'DZA' 'KOR' 'CRI' 'HUN' 'ARE' 'TUN' 'JAM'
 'HRV' 'HKG' 'IRN' 'GEO' 'AND' 'GIB' 'URY' 'JEY' 'CAF' 'CYP' 'COL' 'GGY'
 'KWT' 'NGA' 'MDV' 'VEN' 'SVK' 'FJI' 'KAZ' 'PAK' 'IDN' 'LBN' 'PHL' 'SEN'
 'SYC' 'AZE' 'BHR' 'NZL' 'THA' 'DOM' 'MKD' 'MYS' 'ARM' 'JPN' 'LKA' 'CUB'
 'CMR' 'BIH' 'MUS' 'COM' 'SUR' 'UGA' 'BGR' 'CIV' 'JOR' 'SYR' 'SGP' 'BDI'
 'SAU' 'VNM' 'PLW' 'QAT' 'EGY' 'PER' 'MLT' 'MWI' 'ECU' 'MDG' 'ISL' 'UZB'
 'NPL' 'BHS' 'MAC' 'TGO' 'TWN' 'DJI' 'STP' 'KN

In [7]:
# Replace month names with order number
# Cast the categorical features to the category datatype for the arrival date month
# January -> 1
# Use mapping
month_map = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 
             'July': 7, 'August': 8, "September": 9, 'October': 10, 'November': 11, 'December':21}

# One/Old approach
#cat_df['arrival_date_month']= cat_df['arrival_date_month'].map(month_map)
# brings warning to use .loc

# .loc[rows, columns]
cat_df.loc[:,'arrival_date_month']= cat_df['arrival_date_month'].map(month_map)
cat_df

,hotel,arrival_date_month,meal,country,market_segment,distribution_channel,reserved_room_type,assigned_room_type,deposit_type,customer_type,reservation_status,reservation_status_date
0,Resort Hotel,7,BB,PRT,Direct,Direct,C,C,No Deposit,Transient,Check-Out,2015-07-01
1,Resort Hotel,7,BB,PRT,Direct,Direct,C,C,No Deposit,Transient,Check-Out,2015-07-01
2,Resort Hotel,7,BB,GBR,Direct,Direct,A,C,No Deposit,Transient,Check-Out,2015-07-02
3,Resort Hotel,7,BB,GBR,Corporate,Corporate,A,A,No Deposit,Transient,Check-Out,2015-07-02
4,Resort Hotel,7,BB,GBR,Online TA,TA/TO,A,A,No Deposit,Transient,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,8,BB,BEL,Offline TA/TO,TA/TO,A,A,No Deposit,Transient,Check-Out,2017-09-06
119386,City Hotel,8,BB,FRA,Online TA,TA/TO,E,E,No Deposit,Transient,Check-Out,2017-09-07
119387,City Hotel,8,BB,DEU,Online TA,TA/TO,D,D,No Deposit,Transient,Check-Out,2017-09-07
119388,City Hotel,8,BB,GBR,Online TA,TA/TO,A,A,No Deposit,Transient,Check-Out,2017-09-07


## Label encoding

In [8]:
# Now do the country hotel features
# Use label encoding from sklearn (scikit-learn)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()


In [9]:
# encode country feature
cat_df.loc[:,'country'] = le.fit_transform(cat_df['country'])
cat_df.head()

,hotel,arrival_date_month,meal,country,market_segment,distribution_channel,reserved_room_type,assigned_room_type,deposit_type,customer_type,reservation_status,reservation_status_date
0,Resort Hotel,7,BB,135,Direct,Direct,C,C,No Deposit,Transient,Check-Out,2015-07-01
1,Resort Hotel,7,BB,135,Direct,Direct,C,C,No Deposit,Transient,Check-Out,2015-07-01
2,Resort Hotel,7,BB,59,Direct,Direct,A,C,No Deposit,Transient,Check-Out,2015-07-02
3,Resort Hotel,7,BB,59,Corporate,Corporate,A,A,No Deposit,Transient,Check-Out,2015-07-02
4,Resort Hotel,7,BB,59,Online TA,TA/TO,A,A,No Deposit,Transient,Check-Out,2015-07-03


## One-Hot encoding

In [10]:
# get all your categories
cat_df.columns

Index(['hotel', 'arrival_date_month', 'meal', 'country', 'market_segment',
       'distribution_channel', 'reserved_room_type', 'assigned_room_type',
       'deposit_type', 'customer_type', 'reservation_status',
       'reservation_status_date'],
      dtype='object')

In [12]:
# one hot encoding for ALL features as one
cat_df = cat_df.drop('reservation_status_date', axis =1)
cat_df.columns

Index(['hotel', 'arrival_date_month', 'meal', 'country', 'market_segment',
       'distribution_channel', 'reserved_room_type', 'assigned_room_type',
       'deposit_type', 'customer_type', 'reservation_status'],
      dtype='object')

In [13]:
# one hot encoding for ALL features as 1. Except we fixed 'arrival_date_month' and 'county'
# create a new list of columns with out fixed columns, to pass later

processed_col = ['arrival_date_month','country', 'reservation_status_date']
one_hot_columns = [col for col in cat_df.columns if col not in processed_col]

dummy_df = pd.get_dummies(cat_df, columns=one_hot_columns)
dummy_df.head()

# TODO fix dates as it created too many columns

,arrival_date_month,country,hotel_City Hotel,hotel_Resort Hotel,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,market_segment_Aviation,...,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,reservation_status_Canceled,reservation_status_Check-Out,reservation_status_No-Show
0,7,135,False,True,True,False,False,False,False,False,...,True,False,False,False,False,True,False,False,True,False
1,7,135,False,True,True,False,False,False,False,False,...,True,False,False,False,False,True,False,False,True,False
2,7,59,False,True,True,False,False,False,False,False,...,True,False,False,False,False,True,False,False,True,False
3,7,59,False,True,True,False,False,False,False,False,...,True,False,False,False,False,True,False,False,True,False
4,7,59,False,True,True,False,False,False,False,False,...,True,False,False,False,False,True,False,False,True,False


In [14]:
# merge back to cat_df

cat_df = pd.concat([cat_df, dummy_df], axis=1) # axis =1 apply to columns
cat_df.head()


,hotel,arrival_date_month,meal,country,market_segment,distribution_channel,reserved_room_type,assigned_room_type,deposit_type,customer_type,...,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,reservation_status_Canceled,reservation_status_Check-Out,reservation_status_No-Show
0,Resort Hotel,7,BB,135,Direct,Direct,C,C,No Deposit,Transient,...,True,False,False,False,False,True,False,False,True,False
1,Resort Hotel,7,BB,135,Direct,Direct,C,C,No Deposit,Transient,...,True,False,False,False,False,True,False,False,True,False
2,Resort Hotel,7,BB,59,Direct,Direct,A,C,No Deposit,Transient,...,True,False,False,False,False,True,False,False,True,False
3,Resort Hotel,7,BB,59,Corporate,Corporate,A,A,No Deposit,Transient,...,True,False,False,False,False,True,False,False,True,False
4,Resort Hotel,7,BB,59,Online TA,TA/TO,A,A,No Deposit,Transient,...,True,False,False,False,False,True,False,False,True,False


In [15]:
#drop the original 'object columns

num_df = data_df.drop(columns=cat_cols, axis=1)
num_df.head()

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
0,0,342,2015,27,1,0,0,2,0.0,0,0,0,0,3,NaN,NaN,0,0.0,0,0
1,0,737,2015,27,1,0,0,2,0.0,0,0,0,0,4,NaN,NaN,0,0.0,0,0
2,0,7,2015,27,1,0,1,1,0.0,0,0,0,0,0,NaN,NaN,0,75.0,0,0
3,0,13,2015,27,1,0,1,1,0.0,0,0,0,0,0,304.0,NaN,0,75.0,0,0
4,0,14,2015,27,1,0,2,2,0.0,0,0,0,0,0,240.0,NaN,0,98.0,0,1


In [16]:
final_df = pd.concat([num_df, cat_df], axis=1)

In [17]:
# save to file

final_df.to_csv('../data/booking_processed.csv', index=False) 

## Correlation matrix

In [18]:
corr_matrix = final_df.corr(numeric_only=True)
corr_matrix

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,...,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,reservation_status_Canceled,reservation_status_Check-Out,reservation_status_No-Show
is_canceled,1.000000,0.293123,0.016660,0.008148,-0.006130,-0.001791,0.024765,0.060017,0.005048,-0.032491,...,-0.477911,0.481457,-0.011312,-0.023670,-0.038698,0.133084,-0.124135,0.978435,-1.000000,0.131752
lead_time,0.293123,1.000000,0.040142,0.126871,0.002268,0.085671,0.165799,0.119519,-0.037622,-0.020915,...,-0.380233,0.380179,0.016587,0.068541,-0.031924,-0.173959,0.159548,0.304077,-0.293123,-0.044286
arrival_date_year,0.016660,0.040142,1.000000,-0.540561,-0.000221,0.021497,0.030883,0.029635,0.054624,-0.013192,...,0.065736,-0.065963,-0.000760,-0.171738,-0.010982,0.227954,-0.163588,0.017546,-0.016660,-0.003782
arrival_date_week_number,0.008148,0.126871,-0.540561,1.000000,0.066809,0.018208,0.015558,0.025909,0.005518,0.010395,...,-0.005847,0.007773,-0.016888,0.090334,0.011618,-0.079544,0.042213,0.013112,-0.008148,-0.023591
arrival_date_day_of_month,-0.006130,0.002268,-0.000221,0.066809,1.000000,-0.016354,-0.028174,-0.001566,0.014544,-0.000230,...,0.004978,-0.008616,0.032150,-0.012033,-0.001549,-0.000608,0.006269,-0.009024,0.006130,0.013717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
customer_type_Transient,0.133084,-0.173959,0.227954,-0.079544,-0.000608,0.019485,0.007299,0.091006,0.096135,0.021595,...,-0.109341,0.115867,-0.053428,-0.326153,-0.120893,1.000000,-0.895595,0.131810,-0.133084,0.009876
customer_type_Transient-Party,-0.124135,0.159548,-0.163588,0.042213,0.006269,-0.064920,-0.064281,-0.115760,-0.092846,-0.022925,...,0.114761,-0.121451,0.054654,-0.097061,-0.035977,-0.895595,1.000000,-0.123411,0.124135,-0.006983
reservation_status_Canceled,0.978435,0.304077,0.017546,0.013112,-0.009024,-0.005338,0.023657,0.067409,0.004307,-0.032397,...,-0.486775,0.490337,-0.011076,-0.022348,-0.038464,0.131810,-0.123411,1.000000,-0.978435,-0.075845
reservation_status_Check-Out,-1.000000,-0.293123,-0.016660,-0.008148,0.006130,0.001791,-0.024765,-0.060017,-0.005048,0.032491,...,0.477911,-0.481457,0.011312,0.023670,0.038698,-0.133084,0.124135,-0.978435,1.000000,-0.131752


### Correlation and Causation

**Correlation**: a measure of the  extend of interdependence between variables

**Causation**: the relationship between cause and effect between two variables

#### Pearson correlation

Measures linear dependence between two variables. THe result coefficient is a value between `-1 and +1`, where:

-  `1` perfect positive correlation
- `0` no linear correlation
- `-1` perfect negative correlation

Note: `Pearson` correlation is the default method in pandas for the `corr()` method.

In [19]:
# Do a correlation between 'is_canceled' and 'children' features
corr_matrix_two = final_df[['is_canceled', 'children']].corr()
corr_matrix_two

,is_canceled,children
is_canceled,1.000000,0.005048
children,0.005048,1.000000


Sometimes we want tot know the signoficant of the correlation estimate

#### p-value
The probabilty value that the correlation between the variables is statistically significant
Normally we chose a significance level of `0.05`, which means we are `95%` confident

p-values
- $<$ 0.001: we say it is `strong` evidence
- $<$ 0.05: we say it is `moderate`
- $<$ 0.1 : we say it is `weak`
- $>$ 0.1 : there is `no evidence`

In [22]:
# conda install scipy
from scipy.stats import pearsonr, spearmanr

In [28]:
final_df.head()

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,...,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,reservation_status_Canceled,reservation_status_Check-Out,reservation_status_No-Show
0,0,342,2015,27,1,0,0,2,0.0,0,...,True,False,False,False,False,True,False,False,True,False
1,0,737,2015,27,1,0,0,2,0.0,0,...,True,False,False,False,False,True,False,False,True,False
2,0,7,2015,27,1,0,1,1,0.0,0,...,True,False,False,False,False,True,False,False,True,False
3,0,13,2015,27,1,0,1,1,0.0,0,...,True,False,False,False,False,True,False,False,True,False
4,0,14,2015,27,1,0,2,2,0.0,0,...,True,False,False,False,False,True,False,False,True,False


In [39]:
final_df['children'].fillna(0, inplace=True)

C:\Users\get20\AppData\Local\Temp\ipykernel_37908\677517709.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['children'].fillna(0, inplace=True)


In [42]:
# To avoid warning, need to test
final_df.fillna({'children':0}, inplace=True) 

In [43]:

corr, p_value = pearsonr(final_df['is_canceled'], final_df['children'])
print(f'Correlation: [{corr}]/n P_value:[{p_value}]')

Correlation: [0.005036254836445667]/n P_value:[0.08183043560179469]
